In [1]:
import os
import json

from PIL import Image
BASE_DIR = 'stadium_dataset'
LABELS_DIR = f"{BASE_DIR}/labels"
os.makedirs(LABELS_DIR, exist_ok=True)

with open(f"{BASE_DIR}/annotations/labels.json") as f:
    coco = json.load(f)

class_map = {
    ("hilal", "sitting"): 0,
    ("hilal", "cheering"): 1,
    ("hilal", "fighting"): 2,
    ("hilal", "throwing"): 3,
    ("ittihad", "sitting"): 4,
    ("ittihad", "cheering"): 5,
    ("ittihad", "fighting"): 6,
    ("ittihad", "throwing"): 7
}

# Reverse map for names
names = {v: f"{k[0]}_{k[1]}" for k, v in class_map.items()}

for image in coco['images']:
    img_id = image['id']
    file_name = image['file_name']
    W, H = image['width'], image['height']

    lines = []
    for ann in coco['annotations']:
        if ann['image_id'] != img_id:
            continue

        x, y, w, h = ann['bbox']
        xc = (x + w/2) / W
        yc = (y + h/2) / H
        norm_w = w / W
        norm_h = h / H

        team = ann['attributes']['team']
        action = ann['attributes']['action']
        cls_id = class_map[(team, action)]

        lines.append(f"{cls_id} {xc:.6f} {yc:.6f} {norm_w:.6f} {norm_h:.6f}")

    with open(f"{LABELS_DIR}/{file_name.replace('.png', '.txt')}", 'w') as f:
        f.write("\n".join(lines))


In [2]:
yaml_content = f"""
path: {BASE_DIR}
train: images
val: images

nc: 8
names: {list(names.values())}
"""

with open(f"{BASE_DIR}/dataset.yaml", 'w') as f:
    f.write(yaml_content)


In [4]:
from ultralytics import YOLO

# Create model
model = YOLO('yolov8n.yaml')

# Train
model.train(
    data=f'{BASE_DIR}/dataset.yaml',
    epochs=30,
    imgsz=416,
    batch=16,
    name='fan_detector'
)


Ultralytics 8.3.111  Python-3.12.3 torch-2.6.0+cpu CPU (Intel Core(TM) i7-8665U 1.90GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=stadium_dataset/dataset.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=fan_detector2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes

100%|██████████| 755k/755k [00:00<00:00, 1.57MB/s]


Overriding model.yaml nc=80 with nc=8

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning C:\Users\Adil\AIAssignmets\Assignment9\stadium_dataset\labels... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:01<00:00, 281.47it/s]

train: New cache created: C:\Users\Adil\AIAssignmets\Assignment9\stadium_dataset\labels.cache


...

                   all        500       2712      0.967      0.935      0.982      0.898



30 epochs completed in 1.092 hours.
Optimizer stripped from runs\detect\fan_detector2\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\fan_detector2\weights\best.pt, 6.2MB

Validating runs\detect\fan_detector2\weights\best.pt...
Ultralytics 8.3.111  Python-3.12.3 torch-2.6.0+cpu CPU (Intel Core(TM) i7-8665U 1.90GHz)
YOLOv8n summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:30<00:00,  1.94s/it]


                   all        500       2712      0.965      0.923      0.981      0.934
         hilal_sitting        260        358      0.971      0.922      0.973      0.921
        hilal_cheering        259        346      0.977      0.962      0.988       0.94
        hilal_fighting        244        326      0.952       0.89      0.981      0.946
        hilal_throwing        242        329      0.971      0.909      0.987      0.935
       ittihad_sitting        240        324      0.884       0.96      0.981      0.946
      ittihad_cheering        242        321      0.994      0.959      0.989      0.925
      ittihad_fighting        261        362      0.982       0.89       0.98       0.94
      ittihad_throwing        256        346       0.99      0.896      0.971       0.92
Speed: 1.0ms preprocess, 47.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\fan_detector2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001F066A1DA30>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

In [ ]:
# Load trained model
model = YOLO('runs/detect/fan_detector2/weights/best.pt')

# Predict on sample image
results = model(f'{BASE_DIR}/images/0001.png')

# Show results
results[0].show()

In [7]:
metrics = model.val()
print(metrics.box.map50)  # Mean Average Precision @ IoU=0.5


Ultralytics 8.3.111  Python-3.12.3 torch-2.6.0+cpu CPU (Intel Core(TM) i7-8665U 1.90GHz)
val: Fast image access  (ping: 0.00.0 ms, read: 14.81.3 MB/s, size: 3.0 KB)


val: Scanning C:\Users\Adil\AIAssignmets\Assignment9\stadium_dataset\labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:30<00:00,  1.05it/s]


                   all        500       2712      0.965      0.923      0.981      0.934
         hilal_sitting        260        358      0.971      0.922      0.973      0.921
        hilal_cheering        259        346      0.977      0.962      0.988       0.94
        hilal_fighting        244        326      0.952       0.89      0.981      0.946
        hilal_throwing        242        329      0.971      0.909      0.987      0.935
       ittihad_sitting        240        324      0.884       0.96      0.981      0.946
      ittihad_cheering        242        321      0.994      0.959      0.989      0.925
      ittihad_fighting        261        362      0.982       0.89       0.98       0.94
      ittihad_throwing        256        346       0.99      0.896      0.971       0.92
Speed: 1.1ms preprocess, 45.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\val
0.9811619743712605
